## Import  Libraries

In [1]:
import os 
import imagesize
import zipfile 
import statistics 
import math
import torch
import torchvision
import wandb

import numpy as np
import torch.nn as nn
import torch.cuda as cuda
import torchvision.transforms as T
import torch.nn.functional as F
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt


from tqdm import tqdm
from statistics import mean
from pathlib import Path
from sklearn.model_selection import train_test_split
from torch import Tensor
from torchvision import models
from torch.autograd import Variable
from torch.utils.data.dataloader import DataLoader
from torch.utils.data import Subset
from torch.nn import CrossEntropyLoss
from torch.optim import RMSprop, Adagrad
from overrides import overrides, final
from abc import abstractmethod
#from google.colab import drive




/Users/luciahrovatin/Desktop/deep-learning-proj/venv/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Preprocess Data

In [2]:
#wandb.init(project="uda project",
#           entity="dll_2022")

classes = ["backpack", "bookcase", "car jack", "comb", "crown", "file cabinet", "flat iron", "game controller", "glasses",
           "helicopter", "ice skates", "letter tray", "monitor", "mug", "network switch", "over-ear headphones", "pen",
           "purse", "stand mixer", "stroller"]

domains = ["product_images", "real_life"]

In [3]:
def data_transformation(resize_dim = 256, crop_dim = 224, grayscale = True, crop_center = True):
    
    transform_lst = []
    transform_lst.append(T.Resize((resize_dim)))                                                          
    
    if grayscale:
        transform_lst.append(T.Grayscale(num_output_channels=3))                        
    
    if crop_center:
        transform_lst.append(T.CenterCrop((crop_dim)))
    else:
        transform_lst.append(T.RandomCrop((crop_dim)))
    
    transform_lst.append(T.RandomHorizontalFlip(p=0.5))                                  
    transform_lst.append(T.ToTensor())                                             
        
    return T.Compose(transform_lst)  



In [4]:
def normalization(dataset):
    ds_length = len(dataset)
    for i in tqdm(range(ds_length)):
        r_mean, g_mean, b_mean = torch.mean(dataset[i][0], dim = [1,2])
        r_std, g_std, b_std = torch.std(dataset[i][0], dim = [1,2])
        T.functional.normalize(
            tensor = dataset[i][0], 
            mean = [r_mean, g_mean, b_mean],
            std = [r_std, g_std, b_std],
            inplace=True
            )
    return dataset

In [5]:
source = "product_images"
target = "real_life"
resize_dim = 256
crop_dim = 224
grayscale = False
crop_center = True 


source_ds = torchvision.datasets.ImageFolder(
    root = f"data/Adaptiope/{source}",
    transform = data_transformation(resize_dim, crop_dim, grayscale, crop_center)
    )

target_ds = torchvision.datasets.ImageFolder(
    root = f"data/Adaptiope/{target}",
    transform = data_transformation(resize_dim, crop_dim, grayscale, crop_center)
    ) 

if not grayscale:
    normalization(source_ds)
    normalization(target_ds)
    

100%|██████████| 2000/2000 [01:59<00:00, 16.79it/s]


In [6]:
def get_data(dataset, test_split=0.2, batch_size=32):
    
    train_indices, val_indices = train_test_split(
        list(range(len(dataset.targets))),
        test_size = test_split,
        stratify = dataset.targets, 
        random_state = 42
        )
    
    train_dataset = Subset(dataset, train_indices)
    val_dataset = Subset(dataset, val_indices)

    train_data_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    val_data_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)
    
    return train_data_loader, val_data_loader
    

In [7]:
batch_size = 32
test_split = 0.2

source_train_loader, source_val_loader = get_data(source_ds, test_split, batch_size)
target_train_loader, target_val_loader = get_data(target_ds, test_split, batch_size)

## Losses

In [8]:
class _Loss(nn.Module):
    
    _THRESHOLD = 1e-20
    
    def __init__(self):
        super(_Loss, self).__init__()
        
    def forward(self, input: Tensor):
        prob = self.to_softmax(input)
        return self.loss(prob)
        
    @final
    def add_threshold(self, prob: Tensor):
        '''
        Check whether the probability distribution after the softmax 
        is equal to 0 in any cell. If this holds, a standard threshold
        is added in order to avoid log(0) case. 

        Parameters
        ----------
        prob: Tensor
            output tensor of the softmax operation

        Returns
        -------
        Tensor
            updated tensor (in case the condition above holds)
        '''
        zeros = (prob == 0)
        if torch.any(zeros):
            thre_tensor = torch.zeros(zeros.shape)
            thre_tensor[zeros] = self._THRESHOLD
            prob = prob + thre_tensor
        return prob
    
    def to_softmax(self, features: Tensor):
        '''
        Apply the softmax operation on the features tensor, 
        being the output of a feature extractor. 
        
        Parameters
        ----------
        features: Tensor
            input tensor of the softmax operation

        Returns
        -------
        Tensor
            probability distribution with (possible) threshold
        '''
        prob = F.softmax(features, dim=1)
        return self.add_threshold(prob)
    
    @abstractmethod
    def loss(self, prob: Tensor):
        pass

In [9]:
class EntropyMinimizationLoss(_Loss):
    
    def __init__(self, n_classes: int):
        super(EntropyMinimizationLoss, self).__init__()
        self.n_classes = n_classes
    
    @overrides
    def loss(self, prob: Tensor):
        prob_source = prob[:, :self.n_classes]
        prob_target = prob[:, self.n_classes:]
        prob_sum = prob_source + prob_target
        return -(prob_sum.log().mul(prob_sum).sum(dim=1).mean())

In [10]:
class SplitLoss(_Loss):
    
    def __init__(self, n_classes: int, source: bool, split_first: bool):
        super(SplitLoss, self).__init__()
        self.n_classes = n_classes
        self._is_source = source
        self._split_first = split_first
    
    @overrides
    def to_softmax(self, features: Tensor):
        if self._split_first:
            prob = self.split_vector(features)
            prob = F.softmax(prob, dim=1)
        else:
            prob = F.softmax(features, dim=1)
            prob = self.split_vector(prob)
        return self.add_threshold(prob)
    
    @final
    def split_vector(self, prob: Tensor):
        return prob[:,:self.n_classes] if self._is_source else prob[:,self.n_classes:]

In [11]:
class SplitCrossEntropyLoss(SplitLoss):
    
    def _get_y_labels(self):
        return self._y_labels
    def _set_y_labels(self, y_labels: Variable):
        if not all(y < self.n_classes for y in y_labels):
            raise ValueError('Expected all y labels < n_classes')
        self._y_labels = y_labels
    y_labels = property(fget=_get_y_labels, fset=_set_y_labels)
    
    def __init__(self, n_classes: int, source: bool, split_first: bool):
        super(SplitCrossEntropyLoss, self).__init__(n_classes, source, split_first)
        self.cross_entropy_loss = CrossEntropyLoss()
    
    @overrides
    def loss(self, prob: Tensor):
        '''Computes cross-entropy loss w.r.t. ground-truth (y label)'''
        return self.cross_entropy_loss(prob, self.y_labels)

In [12]:
class DomainDiscriminationLoss(SplitLoss):
    
    def __init__(self, n_classes: int, source: bool):
        super(DomainDiscriminationLoss, self).__init__(n_classes, source, False)
        
    @overrides
    def loss(self, prob: Tensor):
        return -(prob.sum(dim=1).log().mean())

In [13]:
class TrainingObjectives:
    
    @staticmethod
    def domain_discrimination_loss(src_dom_discrim_loss, tgt_dom_discrim_loss):
        return src_dom_discrim_loss + tgt_dom_discrim_loss
    
    @staticmethod
    def category_confusion_loss(src_cat_conf_loss, tgt_cat_conf_loss):
        return 0.5 * (src_cat_conf_loss + tgt_cat_conf_loss)
    
    @staticmethod
    def domain_confusion_loss(src_dom_conf_loss, tgt_dom_conf_loss):
        return 0.5 * (src_dom_conf_loss + tgt_dom_conf_loss)
    
    @staticmethod
    def overall_classifier_loss(src_task_class_loss, tgt_task_class_loss, domain_discrim_loss):
        return src_task_class_loss + tgt_task_class_loss + domain_discrim_loss
    
    @staticmethod
    def overall_generator_loss(cat_conf_loss, dom_conf_loss, tgt_entropy_loss, curr_epoch, tot_epochs):
        lambda_trade_off = 2 / (1 + math.exp(-1 * 10 * curr_epoch / tot_epochs)) - 1
        return cat_conf_loss + lambda_trade_off * (dom_conf_loss + tgt_entropy_loss)

## Feature Extractor

In [14]:

class FeatureExtractor:
    
    def __init__(self, source_only: bool, n_classes: int, freeze = True, n_params_trained = None, model='resnet50', optimizer='rmsprop', lr=0.01, weight_decay=0):
        """_summary_

        Args:
            n_classes (int): number of classes present in the dataset
            n_params_trained (_type_, optional): Number of parameters (i.e., layers to be trained). Defaults to None.
            model (str, optional): Pretrained model to import as feature extractor. Defaults to 'resnet18'.
            optimizer (str, optional): Optimizer for the model. Defaults to 'rmsprop'.
            lr (float, optional): Initial learning rate. Defaults to 0.01.
            weight_decay (int, optional): Initial weight decay. Defaults to 0.
        """
        self.lr = lr 
        
        # Upload pretrained model 
        if model.lower() == 'resnet18': 
            self.model = models.resnet18(pretrained=True)
        elif model.lower() == 'resnet50': 
            self.model = models.resnet50(pretrained=True)
        else:
            raise ValueError('Unknown model')
        
        # Modify last fully-connected layer
        if not source_only: 
            self.model.fc = nn.Linear(
                in_features = self.model.fc.in_features, 
                out_features = n_classes * 2
            )
        else:
            self.model.fc = nn.Linear(
                in_features = self.model.fc.in_features, 
                out_features = n_classes
            ) 
        
        n_params = len(list(self.model.parameters()))
        if n_params_trained is None:
            n_params_trained = n_params
        
        count = 0 
        first_param_trained = n_params - n_params_trained
        
        if freeze:
            for param in self.model.parameters():
                param.requires_grad = (count >= first_param_trained)
                count = count + 1 

            params_to_train = filter(lambda p: p.requires_grad, self.model.parameters())
        
        else:
            
            # Layer-wise Learning Rate Decay (metà gruppi di layers = metà valore del lr)
            
            params_to_train = []
            name_prev_group = None
            
            groups = set([name.split('.')[0] for name, _ in self.model.named_parameters()])
            i = -1 
            for name, param in self.model.named_parameters():
                name_cur_group = name.split('.')[0]
                if name_cur_group != name_prev_group or name_prev_group is None:
                    i = i + 1
                    lr_group = self.decay(i, len(groups)-1)
                name_prev_group = name_cur_group
                params_to_train.append({'params': param, 'lr': lr_group})
            
        # Initialize optimizer
        if optimizer.lower() == 'rmsprop':
            self.optim = torch.optim.RMSprop(
                params = params_to_train,
                lr = lr,
                weight_decay = weight_decay
            )
        elif optimizer.lower() == 'adadelta':
            self.optim = torch.optim.Adadelta(
                params = params_to_train,
                lr = lr,
                weight_decay = weight_decay
            )
        elif optimizer.lower() == 'sgd':
            self.optim = torch.optim.SGD(
                params = params_to_train,
                lr = lr,
                weight_decay = weight_decay,
                nesterov = True
            )
        else:
            raise ValueError('Unknown optimizer')
    
    def decay(self, index: int, n_groups: int):
        sigmoid = lambda x: 1/(1 + np.exp(-x)) 
        return self.lr * sigmoid(10/n_groups * (index - (n_groups/2))) 
            
        

## Model 

In [26]:
class ModelTrainer:
    _INF = 1e20
    
    def __init__(self, model: FeatureExtractor, n_classes: int, epochs: int):
        """Initialize the SymsNet model
        Args:
            model (FeatureExtractor): _description_
            n_classes (int): _description_
            epochs (int): _description_
        """
       
        self.curr_epoch = 0
        self.tot_epochs = epochs
        self.n_classes = n_classes
        self.model = model 
        self.patience = 10 
        
        if cuda.is_available():
            self.model = self.model.cuda()
            # Task classifier losses
            self.src_task_class_loss = SplitCrossEntropyLoss(n_classes=n_classes, source=True, split_first=True).cuda()
            self.tgt_task_class_loss = SplitCrossEntropyLoss(n_classes=n_classes, source=False, split_first=True).cuda()
            # Domain discrimination losses
            self.src_dom_discrim_loss = DomainDiscriminationLoss(n_classes=n_classes, source=True).cuda()
            self.tgt_dom_discrim_loss = DomainDiscriminationLoss(n_classes=n_classes, source=False).cuda()
            # Category-level confusion losses
            self.src_cat_conf_loss = SplitCrossEntropyLoss(n_classes=n_classes, source=True, split_first=False).cuda()
            self.tgt_cat_conf_loss = SplitCrossEntropyLoss(n_classes=n_classes, source=False, split_first=False).cuda()
            # Domain-level confusion losses
            self.src_dom_conf_loss = DomainDiscriminationLoss(n_classes=n_classes, source=True).cuda()
            self.tgt_dom_conf_loss = DomainDiscriminationLoss(n_classes=n_classes, source=False).cuda()
            # Entropy minimization loss
            self.tgt_entropy_loss = EntropyMinimizationLoss(n_classes=n_classes).cuda()    
        
        else: 
            self.src_task_class_loss = SplitCrossEntropyLoss(n_classes=n_classes, source=True, split_first=True)
            self.tgt_task_class_loss = SplitCrossEntropyLoss(n_classes=n_classes, source=False, split_first=True)
            # Domain discrimination losses
            self.src_dom_discrim_loss = DomainDiscriminationLoss(n_classes=n_classes, source=True)
            self.tgt_dom_discrim_loss = DomainDiscriminationLoss(n_classes=n_classes, source=False)
            # Category-level confusion losses
            self.src_cat_conf_loss = SplitCrossEntropyLoss(n_classes=n_classes, source=True, split_first=False)
            self.tgt_cat_conf_loss = SplitCrossEntropyLoss(n_classes=n_classes, source=False, split_first=False)
            # Domain-level confusion losses
            self.src_dom_conf_loss = DomainDiscriminationLoss(n_classes=n_classes, source=True)
            self.tgt_dom_conf_loss = DomainDiscriminationLoss(n_classes=n_classes, source=False)
            # Entropy minimization loss
            self.tgt_entropy_loss = EntropyMinimizationLoss(n_classes=n_classes)   

            
    def train_step(self, X_source: Tensor, y_source: Tensor, X_target: Tensor):
        
        # Tell model go training mode
        self.model.model.train()
        
        # Compute features for both inputs
        X_source_features = self.model.model(X_source)
        X_target_features = self.model.model(X_target)
        
        # Compute overall training objective losses
        classifier_loss, generator_loss = self.overall_losses(X_source_features, X_target_features, y_source)

        # Compute gradients w.r.t. classifier loss
        self.model.optim.zero_grad()
        classifier_loss.backward(retain_graph=True)
        grad_classifier_tmp = []
        for p in self.model.model.parameters():
            if p.grad is not None:
                grad_classifier_tmp.append(p.grad.data.clone())
        
            
        # Compute gradients w.r.t. generator loss
        self.model.optim.zero_grad()
        generator_loss.backward()
        grad_generator_tmp = []
        for p in self.model.model.parameters():
            if p.grad is not None:
                grad_generator_tmp.append(p.grad.data.clone())
    
        count = 0 
        appended = 0 
        n_classification_params = 2 
        n_params = len(list(self.model.model.parameters()))
        for p in self.model.model.parameters():
            if p.grad is not None:
                grad_tmp = p.grad.data.clone()
                grad_tmp.zero_() 
                
                # Whereas the last 2 parameters are trained with classifier loss,
                # the others are trained with the generator loss  
                if count < (n_params - n_classification_params): 
                    grad_tmp = grad_tmp + grad_generator_tmp[appended]
                else: 
                    grad_tmp = grad_tmp + grad_classifier_tmp[appended]
                appended = appended + 1 
                p.grad.data = grad_tmp
            count = count + 1 
        
        # Perform optimizer step    
        self.model.optim.step()
        target = y_source.clone().tolist()
        preds = X_source_features.clone().tolist()
        return classifier_loss, generator_loss, sum(torch.argmax(preds, dim=1) == target)/len(target)
    
    def train_epoch(self, source_dataloader: DataLoader, target_dataloader: DataLoader):
        end_of_epoch = False
        source_batch_loader = enumerate(source_dataloader)
        target_batch_loader = enumerate(target_dataloader)
        gen_loss = []
        cl_loss = []
        acc_source = []
        
        # Train for current epoch
        while not end_of_epoch:
            try:
                # Get next batch for both source and target
                (X_source, y_source) = next(source_batch_loader)[1]
                (X_target, _) = next(target_batch_loader)[1]
                c_loss, g_loss, acc_so = self.train_step(X_source, y_source, X_target)
                gen_loss.append(g_loss), cl_loss.append(c_loss), acc_source.append(acc_so)
                
            # next(iter, default) if default is omitted and iter is empty, a StopIteration exception is yielded
            except StopIteration: 
                end_of_epoch = True
                return mean(cl_loss), mean(gen_loss), max(acc_source) 
            
    def train(self, source_dataloader: DataLoader, target_dataloader: DataLoader):
        
        prev_classifier_loss = self._INF
        prev_generator_loss = self._INF
        prev_acc_source = 0.0
        patience = self.patience 
        
        epoch_iter = tqdm(range(self.tot_epochs), 
                          unit = "epoch",
                          desc = "Training") 
        
        for i in epoch_iter:
        #while self.curr_epoch < self.tot_epochs:
            self.curr_epoch = i
            classifier_loss, generator_loss, acc_source = self.train_epoch(source_dataloader, target_dataloader)
            logs = {
                "classifier_loss": classifier_loss, 
                "generator_loss": generator_loss,
                "accuracy source": acc_source 
                }
            epoch_iter.set_postfix(logs)
            #wandb.watch(self.model)
            # wandb.log(logs)
            
            if (classifier_loss > prev_classifier_loss or 
                generator_loss > prev_generator_loss or 
                acc_source < prev_acc_source):
                patience = patience - 1
                if patience == 0:
                    print('EARLY STOPPING') 
                    break

    
   
    def overall_losses(self, X_source_features, X_target_features, y_source_var) -> tuple[Tensor, Tensor]:
        # Source task classifier loss
        self.src_task_class_loss.y_labels = y_source_var
        _src_task_class_loss = self.src_task_class_loss(X_source_features)
        
        # (Cross-domain) Target task classifier loss
        self.tgt_task_class_loss.y_labels = y_source_var
        _tgt_task_class_loss = self.tgt_task_class_loss(X_source_features)
        
        # Domain discrimination loss
        _src_dom_discrim_loss = self.src_dom_discrim_loss(X_source_features)
        _tgt_dom_discrim_loss = self.tgt_dom_discrim_loss(X_target_features)
        _domain_discrim_loss = TrainingObjectives.domain_discrimination_loss(
            _src_dom_discrim_loss, 
            _tgt_dom_discrim_loss
        )
        
        # Category-level confusion loss
        self.src_cat_conf_loss.y_labels = y_source_var
        self.tgt_cat_conf_loss.y_labels = y_source_var
        _src_cat_conf_loss = self.src_cat_conf_loss(X_source_features)
        _tgt_cat_conf_loss = self.tgt_cat_conf_loss(X_source_features)
        _category_conf_loss = TrainingObjectives.category_confusion_loss(
            _src_cat_conf_loss, 
            _tgt_cat_conf_loss
        )
        
        # Domain-level confusion loss
        _src_dom_conf_loss = self.src_cat_conf_loss(X_target_features)
        _tgt_dom_conf_loss = self.tgt_cat_conf_loss(X_target_features)
        _domain_conf_loss = TrainingObjectives.domain_confusion_loss(
            _src_dom_conf_loss, 
            _tgt_dom_conf_loss
        )

        # Entropy minimization loss
        _tgt_entropy_loss = self.tgt_entropy_loss(X_target_features)
        
        # Overall classifier loss
        _overall_classifier_loss = TrainingObjectives.overall_classifier_loss(
            _src_task_class_loss, 
            _tgt_task_class_loss, 
            _domain_discrim_loss
        )

        # Overall feature extractor loss
        _overall_generator_loss = TrainingObjectives.overall_generator_loss(
            _category_conf_loss, 
            _domain_conf_loss, 
            _tgt_entropy_loss, 
            self.curr_epoch, 
            self.tot_epochs
        )
        
        return _overall_classifier_loss, _overall_generator_loss

In [ ]:
generator = FeatureExtractor(source_only = False, n_classes=20, model='resnet50', freeze = False)
symnet = ModelTrainer(model=generator, n_classes=20, epochs=2)
symnet.train(source_train_loader, target_train_loader)


# FROM THE ASSIGNMENT DESCRIPTION 

Suppose you’re working on the direction product → real world. Then the first thing you will do is train your model on $P_{train}$. Since this is your <i>source domain </i>, you are allowed to use label information (e.g. use a cross entropy loss in your training step). In your test step, you are going to evaluate the model on $RW_{test}$. This will achieve a certain accuracy; since we only trained on the source domain, and not on the target domain, this accuracy refers to the source only scenario. We call it $acc_{so}$. Now you want to evaluate your UDA component which, differently from the former case, implies training on the target domain. Since you are not allowed to use labels there, here you will use any UDA device of your choice. So, in this case, in your training step, you will train supervisedly on $P_{train}$ (like you did before) and simultaneously train unsupervisedly on $RW_train$. In your test step, once again, you want to evaluate on $RW_{test}$. This will achieve a new accuracy $acc_{uda}$, which hopefully will be higher than $acc_{so}$ since this time you also trained on the target domain, even if without label information. At this point you can compute your gain G:
$$G = accu_{da} − acc_{so}$$

In [ ]:
class SourceModelTrainer:
    
    def __init__(self, model: FeatureExtractor, n_classes: int, epochs: int):
        """Initialize the SymsNet model
        Args:
            model (FeatureExtractor): _description_
            n_classes (int): _description_
            epochs (int): _description_
        """
        
        self.curr_epoch = 0
        self.tot_epochs = epochs
        self.n_classes = n_classes
        self.model = model 
        
        if cuda.is_available():
            self.model = self.model.cuda()
        
        # Cross entropy Loss
        self.loss = CrossEntropyLoss()
            
    def train_step(self, X_source: Tensor, y_source: Tensor):
        
        # Tell model go training mode
        self.model.model.train()
        
        # Compute features for both inputs
        X_source_features = self.model.model(X_source)
        
        # Compute overall training objective losses
        general_loss = self.loss(X_source_features, y_source)

        # Compute gradients w.r.t. classifier loss
        self.model.optim.zero_grad()
        general_loss.backward()
        
        # Perform optimizer step    
        self.model.optim.step()
    
    def train_epoch(self, source_dataloader: DataLoader):
        end_of_epoch = False
        source_batch_loader = enumerate(source_dataloader)

        # Train for current epoch
        while not end_of_epoch:
            try:
                # Get next batch for both source and target
                (X_source, y_source) = next(source_batch_loader)[1]
                self.train_step(X_source, y_source)
            # next(iter, default) if default is omitted and iter is empty, a StopIteration exception is yielded
            except StopIteration: 
                end_of_epoch = True
                return
            
    def train(self, source_dataloader: DataLoader):
        prev_general_loss = self._INF
        prev_acc_source = 0.0
        patience = self.patience 
        
        epoch_iter = tqdm(range(self.tot_epochs), 
                          unit = "epoch",
                          desc = "Training source only model") 
        
        for i in epoch_iter:
            general_loss, acc_source = self.train_epoch(source_dataloader)
            logs = {
                "general_loss": general_loss, 
                "accuracy": acc_source 
                }
            epoch_iter.set_postfix(logs)
            
            if (general_loss > prev_general_loss or 
                acc_source < prev_acc_source):
                patience = patience - 1
                if patience == 0:
                    print('EARLY STOPPING') 
                    break

    
   


In [ ]:
source_only_generator = FeatureExtractor(source_only = True, n_classes=20, model='resnet50')
source_only_classifier = SourceModelTrainer(model=generator, n_classes=20, epochs=1)
source_only_classifier.train(source_train_loader)

# Validation

In [ ]:
class ModelValidator: 
    
    def __init__(self, model: FeatureExtractor, n_classes: int, source_only: bool):
        """Initialize the SymsNet model
        Args:
            model (FeatureExtractor): _description_
            n_classes (int): _description_
            epochs (int): _description_
        """
        self.model = model 
        self.n_classes = n_classes
        self.loss_general = CrossEntropyLoss()
        self.loss_source = SplitCrossEntropyLoss(n_classes=n_classes, source=True, split_first=True).cuda()
        self.loss_target = SplitCrossEntropyLoss(n_classes=n_classes, source=False, split_first=True).cuda()
        self.source_only = source_only
        self.val_loss_source = []
        self.val_loss_target = []
        self.acc_uda = []
        
        self.val_loss_general = []
        self.acc_so = []
        
    def validation_step(self, input, target):
        
        with torch.no_grad():
            preds = self.model.model(input)
            
        if self.source_only: 
            loss = self.loss_general(preds, target) 
            acc_so = sum(torch.argmax(preds, dim=1) == target)/len(target)
            return loss, acc_so
        
        else: 
            self.loss_source.y_labels = target
            self.loss_target.y_labels = target
            loss_source = self.loss_source(preds)
            loss_target = self.loss_target(preds)
            acc_uda = sum(torch.argmax(preds, dim=1) == target)/len(target)
            
            return (loss_source, loss_target), acc_uda
        
    
    def validate(self, val_loader: DataLoader):
        
        self.model.model.eval()
        validator = enumerate(val_loader)
        
        for _ , (input, target) in validator: 
            if cuda.is_available():
                input, target = input.cuda(), target.cuda()
            if self.source_only:
                loss, acc  = self.validation_step(input, target)
                self.val_loss_general.append(loss)
                self.acc_so.append(acc)
                 
            else: 
                (ls_source, ls_target), acc = self.validation_step(input, target)
                self.val_loss_source.append(ls_source)
                self.val_loss_target.append(ls_target)
                self.acc_uda.append(acc)
        
        if self.source_only: 
            return min(self.val_loss_general), max(self.acc_so)
        
        return (min(self.val_loss_source), min(self.val_loss_target)), max(self.acc_uda)

In [ ]:
eval_so = ModelValidator(model=source_only_generator, n_classes=20, source_only = True)
val_loss, acc_so = eval_so.validate(target_val_loader)

eval_uda = ModelValidator(model=generator, n_classes=20, source_only = False)
(val_loss_source, val_loss_target), acc_uda =  eval_uda.validate(target_val_loader)

In [ ]:
# Overall GAIN with source only model and UDA model
def overall_gain(acc_so, acc_uda):
    return (acc_uda - acc_so)*100

In [ ]:
print(overall_gain(acc_so, acc_uda))
print(acc_so)
print(acc_uda)

tensor(-3.1250)
tensor(0.1562)
tensor(0.1250)
